# Module 22: FCW + AEB Policy Lab (No-Code)

This module applies **bounded braking** automatically when TTC risk is high.

In [ ]:

import sys, os, time, cv2
from IPython.display import display, Image, clear_output
sys.path.append(os.path.abspath('../'))
from student_api import AIStudentAPI, VESCStudentAPI

ai_api = AIStudentAPI()
if not ai_api.start_camera():
    raise RuntimeError('Camera failed to start. Re-run this cell.')
if not ai_api.start_buzzer():
    raise RuntimeError('Failed to initialize buzzer on GPIO17')

vesc_api = VESCStudentAPI()
vesc = None
if vesc_api.start():
    deadline = time.time() + 10.0
    controllers = []
    while time.time() < deadline and not controllers:
        controllers = vesc_api.get_connected_controllers()
        if not controllers:
            time.sleep(0.5)
    if controllers:
        vesc = vesc_api.get_controller(controllers[0])
if not vesc:
    raise RuntimeError('No VESC controller found. This module requires braking control.')

print('✅ FCW + AEB lab ready')


## Adjustable Policy Variables

In [ ]:

TARGET_LABEL = 'person'
MIN_CONFIDENCE = 0.70

KNOWN_HEIGHT_M = 1.70
FOCAL_PX = 700.0
WHEEL_RADIUS_M = 0.04
GEAR_RATIO = 1.0

TTC_WARNING_S = 3.0
TTC_CRITICAL_S = 1.0

WARNING_BRAKE_A = 2.5
WARNING_BRAKE_RAMP_S = 4.0

CRITICAL_BRAKE_A = 6.0
CRITICAL_BRAKE_RAMP_S = 3.0

ALERT_COOLDOWN_S = 3.0
RUN_DELAY_S = 0.10


In [ ]:

    def rpm_to_speed_mps(rpm):
        wheel_circ = 2.0 * 3.14159 * WHEEL_RADIUS_M
        return abs(rpm) * wheel_circ / 60.0 / max(GEAR_RATIO, 0.01)

    def estimate_distance_m(box_h):
        return (KNOWN_HEIGHT_M * FOCAL_PX) / max(float(box_h), 1.0)

    last_action_time = 0.0

    try:
        print('Running FCW + AEB... press Interrupt (■) to stop.')
        while True:
            frame, detections = ai_api.get_frame_and_detections()
            rpm = vesc.get_rpm() or 0.0
            speed_mps = rpm_to_speed_mps(rpm)

            nearest_dist = float('inf')
            for det in detections:
                x, y, w, h = det['box']
                label = det['label']
                conf = float(det['confidence'])
                cv2.rectangle(frame, (x, y), (x + w, y + h), (4, 23, 115), 2)
                cv2.putText(frame, f"{label}:{conf:.2f}", (x, max(15, y - 5)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (4, 23, 115), 2)
                if label == TARGET_LABEL and conf >= MIN_CONFIDENCE:
                    nearest_dist = min(nearest_dist, estimate_distance_m(h))

            ttc = nearest_dist / speed_mps if speed_mps > 0.05 and nearest_dist < float('inf') else float('inf')

            status = 'SAFE'
            brake_plan = 'None'
            now = time.time()

            if ttc <= TTC_CRITICAL_S:
                status = 'CRITICAL'
                brake_plan = f'{CRITICAL_BRAKE_A:.1f}A/{CRITICAL_BRAKE_RAMP_S:.1f}s'
                if now - last_action_time >= ALERT_COOLDOWN_S:
                    ai_api.buzzer_beep(0.08, 0.08, 3)
                    vesc.set_brake_current(CRITICAL_BRAKE_A, CRITICAL_BRAKE_RAMP_S)
                    last_action_time = now
            elif ttc <= TTC_WARNING_S:
                status = 'WARNING'
                brake_plan = f'{WARNING_BRAKE_A:.1f}A/{WARNING_BRAKE_RAMP_S:.1f}s'
                if now - last_action_time >= ALERT_COOLDOWN_S:
                    ai_api.buzzer_beep(0.12, 0.08, 1)
                    vesc.set_brake_current(WARNING_BRAKE_A, WARNING_BRAKE_RAMP_S)
                    last_action_time = now

            overlay = (
                f"Status={status} | RPM={rpm:.0f} | Speed={speed_mps:.2f} m/s | "
                f"Dist={nearest_dist if nearest_dist < float('inf') else -1:.2f} m | "
                f"TTC={ttc if ttc < float('inf') else -1:.2f} s | BrakePlan={brake_plan}"
            )
            cv2.putText(frame, overlay, (10, 25), cv2.FONT_HERSHEY_SIMPLEX, 0.50, (255, 255, 255), 2)

            _, img_encoded = cv2.imencode('.jpeg', frame)
            clear_output(wait=True)
            display(Image(data=img_encoded.tobytes()))
            print(overlay)

            time.sleep(RUN_DELAY_S)

    except KeyboardInterrupt:
        print('
Stopped.')



## ADAS Knowledge Check (No Coding)

1. Why is bounded brake current safer than max braking every time?
2. What happens if warning and critical thresholds are too close?
3. How does cooldown affect control stability?


## Calibration Module: Tune for Accurate Results (No Coding)

Before you finish this lab, run a calibration pass to make your TTC and brake behavior match reality.

### Step 1: Create a Known-Distance Test Setup
- Place a target object (or person marker) straight ahead of the camera.
- Measure and mark floor distances from the camera: 1 m, 2 m, 3 m, and 4 m.
- Keep camera height and angle fixed for all tests.

### Step 2: Record Vehicle Inputs
Measure or confirm and write down:
- Wheel radius (`WHEEL_RADIUS_M`)
- Gear ratio (`GEAR_RATIO`)
- Typical operating RPM range for your vehicle

### Step 3: Tune Detection Inputs
Adjust and record:
- Target label (`TARGET_LABEL`)
- Confidence threshold (`MIN_CONFIDENCE`)

Goal:
- Stable detection of the intended object
- Low false alerts from unrelated objects

### Step 4: Tune Distance Model Inputs
Adjust and record:
- Known target height (`KNOWN_HEIGHT_M`)
- Focal estimate (`FOCAL_PX`)

Method:
- At each marked floor distance, compare displayed distance vs measured distance.
- Reduce average error across all distance marks.

### Step 5: Tune TTC Policy Inputs
Adjust and record:
- Warning TTC threshold (`TTC_WARNING_S`)
- Critical TTC threshold (`TTC_CRITICAL_S`)

Goal:
- Warning should feel early and informative.
- Critical should trigger only at real near-risk conditions.

### Step 6: Tune Brake Policy Inputs
Adjust and record:
- Warning brake current/time (`WARNING_BRAKE_A`, `WARNING_BRAKE_RAMP_S`)
- Critical brake current/time (`CRITICAL_BRAKE_A`, `CRITICAL_BRAKE_RAMP_S`)
- Alert cooldown (`ALERT_COOLDOWN_S`)

Goal:
- Smooth, predictable intervention in warning state.
- Stronger but still bounded intervention in critical state.

### Step 7: Calibration Record (Fill In)
Record your final tuned values and keep them for deployment:
- `TARGET_LABEL`:
- `MIN_CONFIDENCE`:
- `KNOWN_HEIGHT_M`:
- `FOCAL_PX`:
- `WHEEL_RADIUS_M`:
- `GEAR_RATIO`:
- `TTC_WARNING_S`:
- `TTC_CRITICAL_S`:
- `WARNING_BRAKE_A`:
- `WARNING_BRAKE_RAMP_S`:
- `CRITICAL_BRAKE_A`:
- `CRITICAL_BRAKE_RAMP_S`:
- `ALERT_COOLDOWN_S`:

### Pass Criteria
- Distance estimate is reasonably close at all marked distances.
- Warning and critical transitions are consistent and explainable.
- Brake response is safe, bounded, and repeatable.

In [ ]:

if 'ai_api' in locals():
    ai_api.shutdown_hardware(stop_camera=True, stop_buzzer=True)
    print('Camera + buzzer released.')
if 'vesc_api' in locals() and vesc_api.is_running():
    vesc_api.stop()
    print('VESC API closed.')
